In [123]:
!pip install websocket-client
!pip install json
!pip install numpy
!pip install python-binance
!pip install pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [124]:
##
import websocket, json, pprint, numpy, ta
import pandas as pd
import numpy as np
from websocket import WebSocketApp
import config



## I/ Moyenne du  1 sec bid and ask price

In [125]:
# Stream the market data:

#Carnet ordre pour ask et bid 1s
#BTC-USDT spot:
BTC_USDT = "wss://stream.binance.com:9443/ws/btcusdt@depth@1000ms"
#ETH-USDT spot:
ETH_USDT = "wss://stream.binance.com:9443/ws/ethusdt@depth@1000ms"


# En ce qui concerne les futures, L'api binance ne permet pas d'obtenir l'orderbook à la seconde, le maximum étant 500ms. Nous procéderons donc par un retraitement des messages obtenus toutes les demi-seconde.
#BTC-USD perp:
BTC_USD_p = "wss://fstream.binance.com/ws/btcusdt@depth@500ms"  
#ETH-USD perp:
ETH_USD_p = "wss://fstream.binance.com/ws/ethusdt@depth@500ms"


### Choix des variables d'intérêt

In [126]:
#Variable de choix

#Symbole:
SOCKET= BTC_USDT

#Level of quantity:
target=10


### Définition des fonctions pour le calcul de la moyenne du ask et bid 

In [128]:

#Définition des fonctions:

#Les fonctions s'appliquerons aux données du carnet d'ordre que nous obtiendrons via le websocket et prendrons comme deuxième argument les "targets" [1,10,25]


def Average_ask(data,level_quantity):
    total_quantity_ask= 0  # Quantité accumulée
    total_cost_ask = 0      # Coût total accumulé
   

    #Pour ce faire, nous utiliserons le carnet d'ordre et nous l'utiliserons de sorte à calculer quel est le "prix moyen du panier" qui nous permet d'acheter (au ask) la quantité cible.
    #Itération sur le carnet ordre au ask:
    for price, quantity in data:
        #On commence par le cas où l'offre de ask est soit plus grande que notre target ou fait excéder notre stock. Ainsi on considère une partie de l'offre.
        if total_quantity_ask + quantity > level_quantity:
            #Quantité considérée
            remaining_quantity = level_quantity - total_quantity_ask
            #"Cout" que cela nous rajoute
            total_cost_ask += remaining_quantity * price
            #La quantité totale devient notre quantité cible. On a atteint notre target. Fin de l'iteration
            total_quantity_ask += remaining_quantity
            break
        #Sinon, on "paye" la totalité de l'offre ask.
        else:
            total_cost_ask += price * quantity
            total_quantity_ask += quantity
    
    avg_ask=total_cost_ask / total_quantity_ask if total_quantity_ask > 0 else None
    return avg_ask

#De la même façon nous décrivons la fonction de calcul du bid average:

def Average_bid(data,level_quantity):
    total_quantity_bid = 0  # Quantité accumulée
    total_cost_bid = 0      # Coût total accumulé
    for price, quantity in data:

        if total_quantity_bid + quantity > level_quantity:
            #Quantité considéré
            remaining_quantity = level_quantity - total_quantity_bid
            #"Cout" que cela nous rajoute
            total_cost_bid += remaining_quantity * price
            #La quantité totale devient notre quantité cible. On a atteint notre target. Fin de l'iteration
            total_quantity_bid += remaining_quantity
            break
        else:
            total_cost_bid += price * quantity
            total_quantity_bid += quantity
    
    avg_bid=total_cost_bid / total_quantity_bid if total_quantity_bid> 0 else None
    return avg_bid



### Stream websocket: dictionnaire contenant le ask et bid moyen pour la quantité souhaitée

In [129]:

#Ces objets nous servirons pour les futures. Etant donné que l'api de l'orderbook est fait de telle sorte que nous ne recevons que les données toute les demi-secondes.
#La création de liste et dataframe nous permettra de stocker les informations que l'on reçoit afin la retraiter et calculer l'average bid et ask par seconde.
df=pd.DataFrame()
formatted_times_l=[]
avg_ask_l=[]
avg_bid_l=[]
type_instrument_d=[]
total_l=[]
symbole_d=[]

#Le compteur de message nous servira à afficher l'information 
compteur_messages = 0
messages = [] 

#Configuration du websocket
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global Dictionnaire_ask_bid,compteur_messages
    print('received message')
    json_message = json.loads(message)


    #Création du dictionnaire contenant le bid, ask average.
    Dictionnaire_ask_bid={}



    #Comme dit précédemment nous allons traiter différement les futures et les spots.

    #Pour les futures, l'api de binance ne possède pas l'orderbook à la seconde mais à 500ms nous procéderons donc par moyenne des moyennes:

    if "fstream" in SOCKET:

        #time
        #Transformation de 'E' en date.
        times=pd.to_datetime(json_message['E'],unit='ms')
        formatted_times = times.strftime('%Y-%m-%d %H:%M:%S')

        ##ask bid

        #ask
        data_float_ask = [(float(ask[0]), float(ask[1])) for ask in json_message['a']]

        #bid
        data_float_bid = [(float(bid[0]), float(bid[1])) for bid in json_message['b']]

        #On s'assure que les ask sont ordonnés au prix croissant et bid par prix décroissant.
        data_float_ask = sorted(data_float_ask, key=lambda ask: ask[0])
        data_float_bid = sorted(data_float_bid, key=lambda bid: bid[0], reverse=True)

        #Calcul des averages à l'aide des fonctions définis plus haut:
        avg_ask=Average_ask(data_float_ask,target)
        avg_bid=Average_bid(data_float_bid,target)

        #Symbole
        symbole = json_message['s']
        
        #Ajout des variables aux listes
        formatted_times_l.append(formatted_times)
        avg_ask_l.append(avg_ask)
        avg_bid_l.append(avg_bid)

        #Création du dataframe afin de pourvoir calculer les moyennes par secondes 
        df = pd.DataFrame({'formatted_times': formatted_times_l, 'avg_ask': avg_ask_l, 'avg_bid': avg_bid_l})
        df['formatted_times'] = pd.to_datetime(df['formatted_times'])

        #Maintenant pour avoir notre réponse par seconde, on groupby les informations par secondes et en moyenne. La moyenne du ask par seconde correspondra donc à la moyenne des moyennes du ask lors des deux demi-secondes.
 
        # Grouper par seconde
        df_grouped = df.groupby(df['formatted_times'].dt.floor('S')).agg({      
            'avg_ask': 'mean',
            'avg_bid': 'mean'
        }).reset_index()
        
        #On configure un compteur message de sorte à ne garder l'information que pour les secondes.

        compteur_messages += 1 
        #On ajoute les valeurs souhaitées dans le dictionnaire.
        if compteur_messages % 2 == 0:
        # On réadapte la variable temps à une présentation pour dictionnaire.
            Dictionnaire_ask_bid['time']=df_grouped['formatted_times'].iloc[-1]
            Dictionnaire_ask_bid['time']=Dictionnaire_ask_bid['time'].strftime('%Y-%m-%d %H:%M:%S')

        #On inclus les dernières variables 
            Dictionnaire_ask_bid['Instrument']=json_message['s']
            Dictionnaire_ask_bid['type']="perpetual"
            Dictionnaire_ask_bid['total']=target
            Dictionnaire_ask_bid['avg_bid']=df_grouped['avg_bid'].iloc[-1]
            Dictionnaire_ask_bid['avg_ask']=df_grouped['avg_ask'].iloc[-1]
            
            #On stocke les réponses dans un dictionnaire pourvoir visualiser l'information
            messages.append(Dictionnaire_ask_bid)
            print(Dictionnaire_ask_bid)

    
    else:

        #L'api pour récupérer le carnet d'ordre sur le marché spot autorise le 1s, nous n'avons donc pas de problème de traitement du temps.

        #Time
        #Transformation de 'E' en date.
        times=pd.to_datetime(json_message['E'],unit='ms')
        formatted_times = times.strftime('%Y-%m-%d %H:%M:%S')

        #Average bid and ask:

        #Récupération des données du carnet d'ordre:

        #ask
        data_float_ask = [(float(ask[0]), float(ask[1])) for ask in json_message['a']]

        #bid
        data_float_bid = [(float(bid[0]), float(bid[1])) for bid in json_message['b']]

        #On s'assure que les ask sont ordonnés au prix croissant et bid par prix décroissant.
        data_float_ask = sorted(data_float_ask, key=lambda ask: ask[0])
        data_float_bid = sorted(data_float_bid, key=lambda bid: bid[0], reverse=True)

        #Calcul des averages à l'aide des fonctions definis plus haut:
        avg_ask=Average_ask(data_float_ask,target)
        avg_bid=Average_bid(data_float_bid,target)

        #Ajout des variables dans le dictionnaire 
    
        Dictionnaire_ask_bid['time']=formatted_times
        Dictionnaire_ask_bid['Instrument']=json_message['s']
        Dictionnaire_ask_bid['type']="spot"
        Dictionnaire_ask_bid['total']=target
        Dictionnaire_ask_bid['avg_bid']=avg_bid
        Dictionnaire_ask_bid['avg_ask']=avg_ask

        print(Dictionnaire_ask_bid)

ws = websocket.WebSocketApp(SOCKET,on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()


opened connection
received message
{'time': '2024-03-14 00:51:18', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 71941.65222661998, 'avg_ask': 72807.81433508001}
received message
{'time': '2024-03-14 00:51:19', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 72801.38, 'avg_ask': 72813.97449802005}
received message
{'time': '2024-03-14 00:51:20', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 72809.94565907, 'avg_ask': 72826.69191307997}
received message
{'time': '2024-03-14 00:51:21', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 72809.99, 'avg_ask': 72839.50249188}
received message
{'time': '2024-03-14 00:51:22', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 72809.99, 'avg_ask': 72842.61868455054}
received message
{'time': '2024-03-14 00:51:23', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 10, 'avg_bid': 72809.99, 'avg_ask': 72848.51743856395}
received message
{'time': '2024-03-14 00:51:24

True

### Visualisation du résultat obtenu

In [130]:
# Présentation en vertical

#On distingue deux cas car la fréquence des messages n'est pas la même selon spot ou perpetual.

if "fstream" in SOCKET:
        # dernière information est contenu dans la liste "messages":
        for cle, valeur in messages[-1].items():
            print(f"{cle}: {valeur}")
else: 
        for cle, valeur in Dictionnaire_ask_bid.items():
            print(f"{cle}: {valeur}")


time: 2024-03-14 00:51:26
Instrument: BTCUSDT
type: spot
total: 10
avg_bid: 72787.67575098001
avg_ask: 72811.63130903999


## II/ Création du rapport csv

In [132]:
# Stream the market data:
BTC_USDT_analytics = "wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1s/btcusdt@depth"
ETH_USDT_analytics = "wss://stream.binance.com:9443/stream?streams=ethusdt@kline_1s/ethusdt@depth"
BTC_USD_p_analytics= "wss://fstream.binance.com/stream?streams=btcusdt_perpetual@continuousKline_1m/btcusdt@depth"
ETH_USD_p_analytics= "wss://fstream.binance.com/stream?streams=ethusdt_perpetual@continuousKline_1m/ethusdt@depth"


### Choix de la variable d'intérêt

In [133]:
SOCKET_analytics=ETH_USDT_analytics

### Stream websocket: Création d'un dataframe sur la période de streaming

In [134]:

# Ici encore nous distinguerons spot et future. 
#De plus, nous opérons à présent avec des streams combiné. Le traitement des streams du spot de ne pose pas de problème car ils sont tout les deux réglés sur la seconde.
#Cependant, le traitement des futures est plus délicat car les deux streams sont émis à 250ms.

# Initialisation des listes pour stocker les données
times_l = []  # Liste pour les timestamps
times_l_1=[]
number_trades_l = []
traded_volume_l=[]
quotes_volume_l = []
symbole_l = []
spread_1_l=[]
spread_10_l=[]
spread_25_l=[]
type_instrument_l=[]

#Quant à la variable "Average Volume", nous la construirons nous même en calculant à chaque période la moyenne de l'ensemble des volumes tradés depuis le début du stream:
average_volume_l=[]


#Dans le cas des futures nous définissons donc des variables afin de pouvoir correctement traiter l'information.

# Variables servant au traitement des messages sur la même fréquence.
df=pd.DataFrame()
df_candle=pd.DataFrame()
formatted_times_l=[]
avg_ask_l=[]
avg_bid_l=[]
type_instrument_d=[]
total_l=[]
symbole_d=[]
Dictionnaire_spread_future={}
Dictionnaire_candle_future={}

#Le compteur de message nous servira à afficher l'information. Il nous faut deux types de comptage pour les deux streams
compteur_messages = 0
messages = {}
compteur_bis_messages=0

# Initialisation du DataFrame
analytics = pd.DataFrame()


#Configuration du websocket

def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global analytics,compteur_messages,compteur_bis_messages
    print('received message')
    json_message = json.loads(message)


    data=json_message['data']

    #On distingue deux cas: future ou spot

    if "fstream" in SOCKET_analytics:
    #Comme ici nous utilisons un websocket combiné c'est à dire avec deux types de message (candle et orderbook). Il convient de sélectionner le bon message pour extraire les informations que nous souhaitons.

    #Création des variables spreads grâce à l'orderbook:

        #Identification du message 'orderbook'
        if 'a' in data and 'b' in data :
    
        #Récupération des données du carnet d'ordre:

            #ask
            data_float_ask = [(float(ask[0]), float(ask[1])) for ask in data['a']]

            #bid
            data_float_bid = [(float(bid[0]), float(bid[1])) for bid in data['b']]

            #On s'assure que les ask sont ordonnés au prix croissant et bid par prix décroissant.
            data_float_ask = sorted(data_float_ask, key=lambda ask: ask[0])
            data_float_bid = sorted(data_float_bid, key=lambda bid: bid[0], reverse=True)

            #Calcul des spreads
            spread_1=Average_ask(data_float_ask,1)-Average_bid(data_float_bid,1)
            spread_10=Average_ask(data_float_ask,10)-Average_bid(data_float_bid,10)
            spread_25=Average_ask(data_float_ask,25)-Average_bid(data_float_bid,25)
    
            #ajout à la liste
            spread_1_l.append(float(spread_1))
            spread_10_l.append(float(spread_10))
            spread_25_l.append(float(spread_25))


            #Type de contrat:
            type_instrument_l.append("perpetual")
        
            #Symbole
            symbole = data['s']
            symbole_l.append(symbole)
    
            #Time
            time = pd.to_datetime(data['E'], unit='ms') 
            times_l.append(time)

            #Création un dataframe nous permettant de stocker l'information concernant les spreads.
            df = pd.DataFrame({'formatted_times': times_l, 'spread1': spread_1_l, 'spread10': spread_10_l, 'spread25': spread_25_l})
            df['time'] = pd.to_datetime(df['formatted_times'])

            #On groupby de sorte à calculer la moyenne des spreads par minutes
            df_grouped = df.groupby(df['time'].dt.floor('S')).agg({      
                'spread1': 'mean',
                'spread10': 'mean',
                'spread25': 'mean'
            }).reset_index()

            #On attribue un autre nom
            analytics_spread=df_grouped

            #Ajout du symbole
            analytics_spread['symbole']=data['s']

            #Ajout du contract
            analytics_spread['type']="perpetual"

            #On crée un dictionnaire qui stock les messages des spreads futures que l'on recoit avec websocket
            compteur_messages+=1
            Dictionnaire_spread_future[f'message_{compteur_messages}']=analytics_spread

            #On affiche le dictionnaire
            print(Dictionnaire_spread_future)


        #Identification du message 'candle'
        if 'k' in data:

            #Identification du dictionaire candle
            candle = data['k']

            #Conversion du timestamp en datetime
            time_bis = pd.to_datetime(data['E'], unit='ms') 
            times_l_1.append(time_bis)

            #Identification des autres variables
            number_trades = candle['n']
            quotes_volume = candle['q']
            traded_volume= candle['v']

            #Ajout à la liste
            traded_volume_l.append(float(traded_volume))
            number_trades_l.append(float(number_trades))
            quotes_volume_l.append(float(quotes_volume))
        

            #Cas du volume moyen
            average_volume=np.mean(traded_volume_l)
            average_volume_l.append(float(average_volume))

            #On crée un dataframe nous permettant de stocker l'information concernant la candle.
            df_candle = pd.DataFrame({'formatted_times': times_l_1, 'number of trades': number_trades_l, 'traded volume': traded_volume_l, 'average volume': average_volume_l,'quotes volume': quotes_volume_l,})
            df_candle['time'] = pd.to_datetime(df_candle['formatted_times'])

            #On groupby pour calculer la valeur des variables à la seconde. On somme les variables à sommer (ex: number of trades) et on fait la moyenne pour average volume
            df_grouped_bis = df_candle.groupby(df_candle['time'].dt.floor('S')).agg({      
                'number of trades': 'sum',
                'traded volume': 'sum',
                'average volume': 'mean',
                'quotes volume': 'sum'
            }).reset_index()

            #On renomme le df
            analytics_candle=df_grouped_bis

            #On crée un dictionnaire qui stock les messages des candles des futures que l'on recoit avec websocket.
            compteur_bis_messages+=1
            Dictionnaire_candle_future[f'message_{compteur_messages}']=analytics_candle

            #On affiche le dictionnaire
            print(Dictionnaire_candle_future)

    else:

        #Pour les spots nous n'avons pas besoin de retraiter les valeurs par message reçu car les deux streams sont à la seconde.

        #Identification du message 'orderbook'
        if 'a' in data and 'b' in data :
    
        #Récupération des données du carnet d'ordre:
        #ask
            data_float_ask = [(float(ask[0]), float(ask[1])) for ask in data['a']]

        #bid
            data_float_bid = [(float(bid[0]), float(bid[1])) for bid in data['b']]

        #On s'assure que les ask sont ordonnés au prix croissant et bid par prix décroissant.
            data_float_ask = sorted(data_float_ask, key=lambda ask: ask[0])
            data_float_bid = sorted(data_float_bid, key=lambda bid: bid[0], reverse=True)

        #Calcul des spreads
            spread_1=Average_ask(data_float_ask,1)-Average_bid(data_float_bid,1)
            spread_10=Average_ask(data_float_ask,10)-Average_bid(data_float_bid,10)
            spread_25=Average_ask(data_float_ask,25)-Average_bid(data_float_bid,25)
    
        #ajout à la liste
            spread_1_l.append(float(spread_1))
            spread_10_l.append(float(spread_10))
            spread_25_l.append(float(spread_25))

        #Symbole
            symbole = data['s']
            symbole_l.append(symbole)

        #Time
            time = pd.to_datetime(data['E'], unit='ms') 
            times_l.append(time)
        #Type
            type_instrument_l.append("spot")

    #Création des autres variables d'intérêt
        
    #Identification du message 'candle'
        if 'k' in data:

        #Identification du dictionaire candle
            candle = data['k']

        #Identification des autres variables.
            number_trades = candle['n']
            quotes_volume = candle['q']
            traded_volume= candle['v']

        #ajout à la liste
        
            traded_volume_l.append(float(traded_volume))
            number_trades_l.append(float(number_trades))
            quotes_volume_l.append(float(quotes_volume))
        

            #Cas du volume moyen
            average_volume=np.mean(traded_volume_l)
            average_volume_l.append(float(average_volume))

        # Nous pouvons créer directement le dataframe pour les spots. Ce dernier va s'actualiser à chaque seconde
        analytics = pd.DataFrame({
            'time': times_l,
            'symbole':symbole_l,
            'type':type_instrument_l,
            'number of trades': number_trades_l,
            'traded volume': traded_volume_l,
            'average volume': average_volume_l,
            'quotes volume': quotes_volume_l,
            'spread_1':spread_1_l,
            'spread_10':spread_10_l,
            'spread_25':spread_25_l
        }).set_index('time')  # Définir 'time' comme index
        print(analytics)

ws = websocket.WebSocketApp(SOCKET_analytics, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()
    

opened connection
received message
received message
                         symbole  type  number of trades  traded volume  \
time                                                                      
2024-03-14 00:52:18.310  ETHUSDT  spot               3.0         0.0183   

                         average volume  quotes volume  spread_1  spread_10  \
time                                                                          
2024-03-14 00:52:18.310          0.0183      72.843207      0.01       0.01   

                         spread_25  
time                                
2024-03-14 00:52:18.310       0.01  
received message
received message
                         symbole  type  number of trades  traded volume  \
time                                                                      
2024-03-14 00:52:18.310  ETHUSDT  spot               3.0         0.0183   
2024-03-14 00:52:19.310  ETHUSDT  spot              36.0         9.5586   

                         average volum

True

### Visualisation du résultat obtenu

In [135]:
#La visualisation du code dépend du type d'instrument: spot ou future.

#Pour visualiser les analytics des futures il faut merge les dataframes que nous avons recueillis dans des dictionnnaires pendant le stream. Pour le spot nous visualiserons directement.
if "fstream" in SOCKET_analytics:
    #On sélectionne l'index du dernier dataframe pour chaque dictionnaire
    clé_dernier_analytics_spread=list(Dictionnaire_spread_future.keys())[-1]
    clé_dernier_analytics_candle=list(Dictionnaire_candle_future.keys())[-1]
    #Réarangement du data frame:
    # Enfin on merge les dataframes
    analytics=pd.merge(Dictionnaire_candle_future[clé_dernier_analytics_candle],Dictionnaire_spread_future[clé_dernier_analytics_spread],how='inner',on='time')
    #Réarangement du data frame:
    nouvel_ordre = ['symbole', 'type'] + [col for col in analytics.columns if col not in ['symbole', 'type']]
    analytics=analytics[nouvel_ordre].set_index('time')

else:
    #On nettoie l'index en enlevant les millisecondes présents dans l'index du spot analytics:
    analytics.index = analytics.index.floor('S')

analytics

,symbole,type,number of trades,traded volume,average volume,quotes volume,spread_1,spread_10,spread_25
time,,,,,,,,,
2024-03-14 00:52:18,ETHUSDT,spot,3.0,0.0183,0.018300,72.843207,0.01,0.010000,0.010000
2024-03-14 00:52:19,ETHUSDT,spot,36.0,9.5586,4.788450,38048.063605,0.01,0.010000,0.021792
2024-03-14 00:52:20,ETHUSDT,spot,24.0,1.0741,3.550333,4275.098178,0.01,0.027486,0.100549
2024-03-14 00:52:21,ETHUSDT,spot,7.0,0.3013,2.738075,1199.185252,0.01,0.208631,0.524516
2024-03-14 00:52:22,ETHUSDT,spot,6.0,0.0435,2.199160,173.130113,0.01,0.010000,0.057853
2024-03-14 00:52:23,ETHUSDT,spot,2.0,0.0074,1.833867,29.452074,0.01,0.010000,0.132265
2024-03-14 00:52:24,ETHUSDT,spot,4.0,0.2006,1.600543,798.389128,0.01,0.010000,0.010000
2024-03-14 00:52:25,ETHUSDT,spot,4.0,1.0835,1.535913,4312.338050,0.01,0.010000,0.039921
2024-03-14 00:52:26,ETHUSDT,spot,5.0,1.0155,1.478089,4041.690413,0.01,0.010000,0.010000


### Transformation du dataframe en fichier csv

In [136]:
# Transformer le data frame de la fin du stream en csv.

#Parametrisation du fichier csv:
symbole=analytics['symbole'].unique()[0]
contract=analytics['type'].unique()[0]
filename = f'analytics_instrument_{symbole}_{contract}.csv'

#Conversion du data frame en fichier csv
analytics.to_csv(filename)

### Visualisation du rapport csv

In [137]:
#Visualisation du fichier csv
Analytics_csv = pd.read_csv(filename, index_col='time')
Analytics_csv

,symbole,type,number of trades,traded volume,average volume,quotes volume,spread_1,spread_10,spread_25
time,,,,,,,,,
2024-03-14 00:52:18,ETHUSDT,spot,3.0,0.0183,0.018300,72.843207,0.01,0.010000,0.010000
2024-03-14 00:52:19,ETHUSDT,spot,36.0,9.5586,4.788450,38048.063605,0.01,0.010000,0.021792
2024-03-14 00:52:20,ETHUSDT,spot,24.0,1.0741,3.550333,4275.098178,0.01,0.027486,0.100549
2024-03-14 00:52:21,ETHUSDT,spot,7.0,0.3013,2.738075,1199.185252,0.01,0.208631,0.524516
2024-03-14 00:52:22,ETHUSDT,spot,6.0,0.0435,2.199160,173.130113,0.01,0.010000,0.057853
2024-03-14 00:52:23,ETHUSDT,spot,2.0,0.0074,1.833867,29.452074,0.01,0.010000,0.132265
2024-03-14 00:52:24,ETHUSDT,spot,4.0,0.2006,1.600543,798.389128,0.01,0.010000,0.010000
2024-03-14 00:52:25,ETHUSDT,spot,4.0,1.0835,1.535913,4312.338050,0.01,0.010000,0.039921
2024-03-14 00:52:26,ETHUSDT,spot,5.0,1.0155,1.478089,4041.690413,0.01,0.010000,0.010000
